In [1]:
import glob
import csv
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import swifter
import pickle
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
riders = pd.read_csv('./riders_clean.csv')

c:\Users\20228\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
riders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29189843 entries, 0 to 29189842
Data columns (total 26 columns):
 #   Column              Dtype  
---  ------              -----  
 0   index               int64  
 1   ride_id             object 
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  year                int64  
 15  start_date          object 
 16  stop_date           object 
 17  start_hour          int64  
 18  stop_hour           int64  
 19  start_min           int64  
 20  stop_min            int64  
 21  month               int64  
 22  season              int64  
 23  dayofweek           int64  
 24  tripduration        fl

In [7]:
riders_clean_1=riders.copy()

In [21]:
outgoing_rides = riders_clean_1.groupby(['start_station_id','start_date','season','dayofweek','start_hour'])[['ride_id']].count().reset_index()

In [22]:
outgoing_rides.rename(columns={'ride_id':'request'},inplace=True)

In [23]:
outgoing_rides.describe()

,season,dayofweek,start_hour,request
count,8.027124e+06,8.027124e+06,8.027124e+06,8.027124e+06
mean,2.544701e+00,2.961180e+00,1.336175e+01,3.636401e+00
std,1.061244e+00,1.969857e+00,5.784334e+00,4.480496e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00,9.000000e+00,1.000000e+00
50%,2.000000e+00,3.000000e+00,1.400000e+01,2.000000e+00
75%,4.000000e+00,5.000000e+00,1.800000e+01,4.000000e+00
max,4.000000e+00,6.000000e+00,2.300000e+01,1.390000e+02


In [33]:
outgoing_rides['bike_demand'] = outgoing_rides['request'].apply(lambda x: 'High' if x > 4 else \
'Medium' if x >=2  else 'Low')

In [34]:
outgoing_rides.to_csv("./outgoing.csv")

In [30]:
incoming_rides = riders_clean_1.groupby(['end_station_id','stop_date','season','dayofweek','stop_hour'])[['ride_id']].count().reset_index()
incoming_rides.rename(columns={'ride_id':'return'},inplace=True)

In [35]:
incoming_rides.describe()

,season,dayofweek,stop_hour,return
count,8.163854e+06,8.163854e+06,8.163854e+06,8.163854e+06
mean,2.592298e+00,2.976548e+00,1.338976e+01,3.575498e+00
std,1.045763e+00,1.970589e+00,6.002808e+00,4.420470e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00,9.000000e+00,1.000000e+00
50%,3.000000e+00,3.000000e+00,1.400000e+01,2.000000e+00
75%,3.000000e+00,5.000000e+00,1.800000e+01,4.000000e+00
max,4.000000e+00,6.000000e+00,2.300000e+01,1.140000e+02


In [36]:
incoming_rides['return_demand'] = incoming_rides['return'].apply(lambda x: 'High' if x > 6 else \
'Medium' if x >= 2 else 'Low')

In [37]:
incoming_rides.to_csv("./incoming.csv")